# **1. Чтение данных из файла**

Определю количество строк и столбцов, подсчитаю число пользователей и число фильмов

In [148]:
import pandas as pd
import numpy as np
from sklearn import model_selection as cv
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.spatial import distance
from  sklearn.metrics.pairwise import pairwise_distances

df = pd.read_csv('ratings.csv')
df = df.drop(['timestamp'], axis = 1)
print('Количество строк и столбцов: {}'.format(df.shape), "\n")

n_users = len(df['userId'].unique())
n_movies = len(df['movieId'].unique())
print("Число пользователей:", n_users,"\n", "Число фильмов:", n_movies, "\n")
df.head()

Количество строк и столбцов: (99983, 3) 

Число пользователей: 702 
 Число фильмов: 8227 



,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


# **2.Тренировочная и тестовая выборка**

In [149]:
train_data, test_data = cv.train_test_split(df, test_size=0.2)

print('Тренировочная выборка: {}'.format(train_data.shape))
print('Тестовая выборка: {}'.format(test_data.shape))

Тренировочная выборка: (79986, 3)
Тестовая выборка: (19997, 3)


# **3. Матрицы**

In [150]:
train_data_matrix = train_data.pivot(index='userId', columns='movieId', values='rating').fillna(0).values
test_data_matrix = test_data.pivot(index='userId', columns='movieId', values='rating').fillna(0).values

print(test_data_matrix)

[[0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [3.5 0.  0.  ... 0.  0.  0. ]]


# **4. Косинусное расстояние между юзерами**

In [151]:
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
print(user_similarity)

[[0.         0.94286881 0.79987169 ... 0.8321968  0.95750174 0.85150234]
 [0.94286881 0.         0.91080629 ... 0.9550165  0.93109106 0.93153371]
 [0.79987169 0.91080629 0.         ... 0.79120422 0.96870981 0.75279749]
 ...
 [0.8321968  0.9550165  0.79120422 ... 0.         0.90199113 0.83114197]
 [0.95750174 0.93109106 0.96870981 ... 0.90199113 0.         0.9325615 ]
 [0.85150234 0.93153371 0.75279749 ... 0.83114197 0.9325615  0.        ]]


# **5. Коллаборативная фильтрация и RMSE**

In [152]:
def rmse(prediction, ground_truth): #RMSE - оценка качества рекомендации
    prediction = np.nan_to_num(prediction)[ground_truth.nonzero()].flatten()
    ground_truth = np.nan_to_num(ground_truth)[ground_truth.nonzero()].flatten()

    mse = mean_squared_error(prediction, ground_truth)
    return sqrt(mse)


def naive_predict(top):

    top_sim_users = []

    #получаю top похожих пользователей
    for i in range(user_similarity.shape[0]):
        # Получаем индексы топ-N пользователей, исключая самого пользователя (i)
        top_users = user_similarity[i].argsort()[::-1][1:top + 1]
        top_sim_users.append(top_users)

    n_users, n_movies = train_data_matrix.shape
    top_similar_ratings = np.zeros((n_users, len(top_sim_users[0]), n_movies))  # Массив для оценок


    # Извлекаю оценки фильмов для топ похожих пользователей
    for i, similar_users in enumerate(top_sim_users):
        top_similar_ratings[i] = train_data_matrix[similar_users]

    pred = np.zeros((n_users, n_movies)) #матрица предсказанных значений

    #реализация формулы
    for i in range(n_users):
        pred[i] = top_similar_ratings[i].sum(axis=0) / top

    return pred

naive_pred = naive_predict(10)
print('User-based проверка RMSE: ', rmse(naive_pred, test_data_matrix))

User-based проверка RMSE:  3.6527351219436457
